# CodeGen predicted solutions

In [1]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML

GENERATED_ROOT_PATH = os.path.join("..", "input", "bugnet")
GENERATED_PAIRS_PATH = os.path.join(GENERATED_ROOT_PATH, "generated_pairs.csv")
CODEGEN_PAIRS_PATH = os.path.join(GENERATED_ROOT_PATH, "codegen_results.csv")

In [2]:
df = pd.read_csv(CODEGEN_PAIRS_PATH, keep_default_na=False)

df.head()

,problem_id,language,original_status,original_src,changed_src,change,i1,i2,j1,j2,error,stderr,predicted,execute_output
0,p00001,C++,Runtime Error,#include <algorithm>\n#include <cstdio>\n\nusi...,#include <algorithm>\n#include <cstdio>\n\nusi...,replace,11,12,11,12,-11,,\n#include <algorithm>\n#include <cctype>\n#in...,
1,p00001,C++,Time Limit Exceeded,#include <algorithm>\n#include <bitset>\n#incl...,#include <algorithm>\n#include <bitset>\n#incl...,replace,35,38,35,36,TLE,,\n#include <algorithm>\n#include <iostream>\n#...,
2,p00001,C++,Time Limit Exceeded,#include <algorithm>\n#include <iostream>\n#in...,#include <algorithm>\n#include <iostream>\n#in...,replace,12,13,12,13,TLE,,\n#include <algorithm>\n#include <iostream>\n#...,
3,p00001,C++,Runtime Error,#include <algorithm>\n#include <iostream>\nusi...,#include <algorithm>\n#include <iostream>\nusi...,replace,9,10,9,10,-11,,\n#include <algorithm>\n#include <iostream>\nu...,
4,p00001,C++,Time Limit Exceeded,#include <algorithm>\n#include <iostream>\n\nu...,#include <algorithm>\n#include <iostream>\n\nu...,replace,9,14,9,10,TLE,,\n#include <algorithm>\n#include <iostream>\n\...,


In [3]:
df["correct_exact"] = df["predicted"] == df["changed_src"]

correct = df["correct_exact"].sum()
print(f"The exact accuracy of the codegen model is {correct / len(df)}")

The exact accuracy of the codegen model is 0.0


In [4]:
DERIVED_PATH = os.path.join("..", "input", "Project_CodeNet", "derived")

def id2inout(problem_id: str, name: str = "input") -> str:
    return os.path.join(DERIVED_PATH, "input_output", "data", problem_id, f"{name}.txt")

def check(row: pd.Series) -> bool:
    with open(id2inout(row["problem_id"], name="output"), "r") as f:
        output = f.read()

    return output == row["execute_output"]

df["correct_execute"] = df.apply(check, axis="columns")

correct = df["correct_execute"].sum()
print(f"The execute accuracy of the codegen model is {correct / len(df)}")

The execute accuracy of the codegen model is 0.02


In [5]:
lang_df = df.groupby("language")["correct_exact"].agg(["sum", "count"])
lang_df["accuracy"] = lang_df["sum"] / lang_df["count"]

print("Exact match accuracy")
lang_df

Exact match accuracy


,sum,count,accuracy
language,,,
C++,0,100,0.0
Python,0,100,0.0


In [6]:
lang_df = df.groupby("language")["correct_execute"].agg(["sum", "count"])
lang_df["accuracy"] = lang_df["sum"] / lang_df["count"]

print("Execute accuracy")
lang_df

Execute accuracy


,sum,count,accuracy
language,,,
C++,0,100,0.00
Python,4,100,0.04


In [7]:
def get_bug_type(row: pd.Series) -> str:
    line = "\n".join(row["changed_src"].splitlines()[row["j1"] : row["j2"]])
    language = row["language"]

    if language == "Python":
        return (
            "input"
            if "input" in line
            else "output"
            if "print" in line
            else "algorithm"
        )

    if language == "C++":
        return (
            "input"
            if ("cin" in line or "scanf" in line)
            else "output"
            if ("cout" in line or "printf" in line)
            else "algorithm"
        )

    raise NotImplementedError(f"{language} not implemented yet")

df["type"] = df.apply(get_bug_type, axis="columns")

In [8]:
lang_df = df.groupby(["language", "type"])["correct_exact"].agg(
    ["sum", "count"]
)
lang_df["accuracy"] = lang_df["sum"] / lang_df["count"]

print("Exact match accuracy")
lang_df

Exact match accuracy


sum  count  accuracy
language type                           
C++      algorithm    0     26       0.0
         input        0     70       0.0
         output       0      4       0.0
Python   algorithm    0     17       0.0
         input        0     72       0.0
         output       0     11       0.0

In [9]:
lang_df = df.groupby(["language", "type"])["correct_execute"].agg(
    ["sum", "count"]
)
lang_df["accuracy"] = lang_df["sum"] / lang_df["count"]

print("Execute accuracy")
lang_df

Execute accuracy


sum  count  accuracy
language type                           
C++      algorithm    0     26  0.000000
         input        0     70  0.000000
         output       0      4  0.000000
Python   algorithm    2     17  0.117647
         input        2     72  0.027778
         output       0     11  0.000000

In [10]:
def color_source(source_code: str, i1: int, i2: int, color: str='red'):
    lines = source_code.splitlines(keepends=True)
    
    text = ""
    for i, line_str in enumerate(lines):
        for char in line_str:
            norm_color = 'black'
            if char == ' ':
                char = "•"
                norm_color = 'lightgrey'
            if char == '\n':
                char = "↵\n"
                norm_color = 'lightgrey'
            text += f'<span style="color:{color if i1 <= i and i <= i2 else norm_color};">{char}</span>'

    return "<pre>" + text + "</pre>"

In [11]:
df["original_src_html"] = df.apply(
    lambda row: color_source(row["original_src"], row["i1"], row["i2"], color="red"), 
    axis="columns",
)

df["changed_src_html"] = df.apply(
    lambda row: color_source(row["changed_src"], row["j1"], row["j2"], color="green"), 
    axis="columns",
)

df["src_html"] = df.apply(
    lambda row: color_source(row["predicted"], -1, -1, color="green"), 
    axis="columns",
)

In [12]:
correct_df = df[df["correct_exact"]]
for i in range(min(len(correct_df), 10)):
    display(HTML(f"<h2>Example {i}</h2>"))
    
    display(HTML(f"<h6>Original Source Code</h6>"))
    display(HTML(correct_df["original_src_html"].iloc[i]))

    display(HTML(f"<h6>Changed Source Code</h6>"))
    display(HTML(correct_df["changed_src_html"].iloc[i]))

    display(HTML(f"<h6>CodeGen Prediction</h6>"))
    display(HTML(correct_df["src_html"].iloc[i]))

In [13]:
correct_df = df[df["correct_execute"]]
for i in range(min(len(correct_df), 10)):
    display(HTML(f"<h2>Example {i}</h2>"))
    
    display(HTML(f"<h6>Original Source Code</h6>"))
    display(HTML(correct_df["original_src_html"].iloc[i]))

    display(HTML(f"<h6>Changed Source Code</h6>"))
    display(HTML(correct_df["changed_src_html"].iloc[i]))

    display(HTML(f"<h6>CodeGen Prediction</h6>"))
    display(HTML(correct_df["src_html"].iloc[i]))

In [14]:
for i in range(min(len(df), 10)):
    display(HTML(f"<h2>Example {i}</h2>"))
    
    display(HTML(f"<h6>Original Source Code</h6>"))
    display(HTML(df["original_src_html"].iloc[i]))

    display(HTML(f"<h6>Changed Source Code</h6>"))
    display(HTML(df["changed_src_html"].iloc[i]))

    display(HTML(f"<h6>CodeGen Prediction</h6>"))
    display(HTML(df["src_html"].iloc[i]))